In [1]:
import os
from xml.etree.ElementTree import parse
from glob import glob

In [2]:
infile='xray_data/ecgen-radiology/2263.xml'

In [3]:
doc = parse(infile)

In [4]:
node_root = doc.getroot()

In [5]:
# dir(node_root)

In [6]:
for node_x in node_root.iterfind('MedlineCitation/Article/Abstract/AbstractText'):
    print(node_x)
    if node_x.get('Label') == 'FINDINGS':
        print(node_x.text)

<Element 'AbstractText' at 0x7f72905b5048>
<Element 'AbstractText' at 0x7f72905b5098>
<Element 'AbstractText' at 0x7f72905b5138>
Heart size moderately enlarged, stable mediastinal contours. XXXX XXXX opacity in the left lung base. Otherwise, no focal alveolar consolidation, no definite pleural effusion seen. No typical findings of pulmonary edema.
<Element 'AbstractText' at 0x7f72905b5188>


In [7]:
node_root.find('MedlineCitation/Article/Abstract/AbstractText')

<Element 'AbstractText' at 0x7f72905b5048>

In [8]:
node_root.find('parentImage').get('id')

'CXR2263_IM-0853-1001'

In [9]:
zhengwei_file = './xray_data/zhengwei.txt'
zhengwei_list = []
with open(zhengwei_file, 'r') as f:
    for line in f.readlines():
        line = line.strip()
        if line is None or len(line) == 0:
            continue
        zhengwei_list.append(line.split('.')[0])

In [10]:
# begin ['.',',','[','(']
# end [';', '?', ':', '.', ',', ')', '/']
def replace_word(inword):
    beg_c = ['.',',','[','(']
    end_c = [';', '?', ':', '.', ',', ')', '/']
    if len(inword) == 1:
        return inword
    if inword[0] in beg_c:
        inword = inword[0] + ' ' + inword[1:]
    if inword[-1] in end_c:
        inword = inword[:-1]+' '+inword[-1]
    return inword

In [11]:
# xml 中的数据读取到txt中，每行的格式为 “findings\timage_id0\timage_id1\timage_id2\t...”
def gen_report_txt(report_in):
    doc = parse(report_in)
    node_root = doc.getroot()
    findings = ''
    images = []
    try:
        for node_x in node_root.iterfind('MedlineCitation/Article/Abstract/AbstractText'):
            if node_x.get('Label') == 'FINDINGS':
                findings = node_x.text
                if findings is None:
                    return None
                ss = findings.split()
                ss1 = [replace_word(i) for i in ss]
                findings = ' '.join(ss1)
        for node_x in node_root.iterfind('parentImage'):
            img_id = node_x.get('id')
            if img_id in zhengwei_list:
                images.append(node_x.get('id'))
        
        out = '{}'.format(findings)
        for img in images:
            if img is None:
                return None
            out += '\t{}'.format(img)
    except:
        return None
    return out

In [12]:
indir = './xray_data/ecgen-radiology/'
xml_list = glob(os.path.join(indir, '*.xml'))
out_list = [gen_report_txt(i) for i in xml_list]

In [13]:
out_list[:10]

['Heart size moderately enlarged , stable mediastinal contours . XXXX XXXX opacity in the left lung base . Otherwise , no focal alveolar consolidation , no definite pleural effusion seen . No typical findings of pulmonary edema .\tCXR2263_IM-0853-1001',
 None,
 'Normal heart size . Stable tortuous thoracic aorta . Prior granulomatous disease . Healed rib fractures appear stable . Focal opacity is noted in the left midlung overlying the 9th posterior rib which XXXX represents healing rib callus . No pneumothorax or pleural effusion .\tCXR2138_IM-0760-1001',
 'Heart size is moderately enlarged . The pulmonary XXXX and mediastinum are within normal limits . There is no pleural effusion or pneumothorax . There is suspected right lower lobe airspace opacity XXXX demonstrated on the lateral study . There is a fracture of superior sternotomy XXXX unchanged .',
 'Heart size within normal limits , stable mediastinal and hilar contours . Stable mild hyperinflation , right apical pleural-parenchy

In [14]:
out_list_norm = [i for i in out_list if i is not None]

In [15]:
len(out_list)

3955

In [16]:
len(out_list_norm)

3425

In [17]:
with open('./xray_data/findings1.txt', 'w') as f:
    f.write('\n'.join(out_list_norm))

In [18]:
ls ./xray_data/

data/      ecgen-radiology/  findings1.txt  NLMCXR_png.tgz      vocab/
data_256/  error.txt         findings.txt   NLMCXR_reports.tgz  zhengwei.txt
data_512/  features/         lateral.txt    Thumbs.db
